In [2]:
# Import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Write "%matplotlib inline" for jupyter
import seaborn as sns
import csv
import warnings
warnings.filterwarnings('ignore')

#Import train dataset

df = pd.read_csv('train.csv')

# print(df.head())  
# print(df.shape)
# print(df.dtypes)

# Dropping unwanted columns

df = df.drop('Name', axis=1)
df = df.drop('Ticket', axis=1)
df = df.drop('Fare', axis=1)
df = df.drop('Cabin', axis=1)

# Let us combine number of sibling/spouse and number of parents/children

df['Family'] = df['SibSp'] + df['Parch'] + 1
# Drop used features no longer required

df = df.drop('SibSp', axis=1)
df = df.drop('Parch', axis=1)

# print (df.describe())
# count in Age is 714 so there are NaNs

df['Age'] = df['Age'].fillna(df['Age'].median())
df.describe()





,PassengerId,Survived,Pclass,Age,Family
count,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.361582,1.904602
std,257.353842,0.486592,0.836071,13.019697,1.613459
min,1.000000,0.000000,1.000000,0.420000,1.000000
25%,223.500000,0.000000,2.000000,22.000000,1.000000
50%,446.000000,0.000000,3.000000,28.000000,1.000000
75%,668.500000,1.000000,3.000000,35.000000,2.000000
max,891.000000,1.000000,3.000000,80.000000,11.000000


In [3]:
df['Embarked'].value_counts().sum()

889

This means Embarked has 2 NaNs

We fill the mode in place of NaNs

In [4]:
df['Embarked'].mode()

0    S
Name: Embarked, dtype: object

In [5]:
df['Embarked'] = df['Embarked'].fillna('S')
df['Embarked'].describe()

count     891
unique      3
top         S
freq      646
Name: Embarked, dtype: object

Converting categorical variables to numerical variables

In [6]:
df.Embarked.replace(['S','C','Q'],[1,2,3], inplace=True)
df.Sex.replace(['male','female'],[1,0], inplace=True)

In [7]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,Embarked,Family
0,1,0,3,1,22.0,1,2
1,2,1,1,0,38.0,2,2
2,3,1,3,0,26.0,1,1
3,4,1,1,0,35.0,1,2
4,5,0,3,1,35.0,1,1


No further numerical changes can be made without altering the data

# Applying KNN(Simple Fold & Cross Fold)

In [8]:
pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from collections import Counter
from sklearn.model_selection import cross_validate

In [10]:
X = np.array(df.filter(['Pclass','Sex','Embarked','Family','Age'], axis=1))

In [11]:
y = np.array(df.filter(['Survived'], axis=1))

In [12]:
# Simple Cross Validation

X_1, X_test, y_1, y_test = train_test_split(X,y, test_size = 0.3)
X_tr, X_cv, y_tr, y_cv = train_test_split(X_1, y_1, test_size = 0.3)

In [13]:
final_scores = []
for i in range(1,30,2):
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(X_tr, y_tr)
    pred = knn.predict(X_cv)
    acc = accuracy_score(y_cv, pred, normalize = True) * float(100)
    final_scores.append(acc)
    print('\n CV accuracy for k=%d is %d'%(i,acc))


 CV accuracy for k=1 is 79

 CV accuracy for k=3 is 75

 CV accuracy for k=5 is 73

 CV accuracy for k=7 is 72

 CV accuracy for k=9 is 73

 CV accuracy for k=11 is 72

 CV accuracy for k=13 is 76

 CV accuracy for k=15 is 74

 CV accuracy for k=17 is 73

 CV accuracy for k=19 is 71

 CV accuracy for k=21 is 69

 CV accuracy for k=23 is 67

 CV accuracy for k=25 is 65

 CV accuracy for k=27 is 67

 CV accuracy for k=29 is 67


In [22]:
optimal_k = final_scores.index(max(final_scores)) + 1
print(optimal_k)

1


In [23]:
df_test = pd.read_csv('test.csv')
df_test = df_test.drop('Name', axis=1)
df_test = df_test.drop('Ticket', axis=1)
df_test = df_test.drop('Fare', axis=1)
df_test = df_test.drop('Cabin', axis=1)
df_test['Family'] = df_test['SibSp'] + df_test['Parch'] + 1
df_test = df_test.drop('SibSp', axis=1)
df_test = df_test.drop('Parch', axis=1)
df_test["Age"] = df_test["Age"].fillna(df["Age"].median())

In [24]:
df_test1 = pd.read_csv('test.csv')
df_test1

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [25]:
df_test["Embarked"].mode()

0    S
Name: Embarked, dtype: object

In [26]:
df_test['Embarked'] = df_test['Embarked'].fillna('S')

In [27]:
df_test.Embarked.replace(['S','C','Q'], [1,2,3], inplace=True)
df_test.Sex.replace(['male','female'],[1,0], inplace=True)

In [28]:
X_test = np.array(df_test.filter(['Pclass','Sex', 'Embarked', 'Family', 'Age'], axis=1))
knn = KNeighborsClassifier(n_neighbors = optimal_k)
knn.fit(X_tr, y_tr)
pred = knn.predict(X_test)
print(pred)

[0 0 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1 0 0 0 1 0 0 0 0
 1 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0
 1 1 0 1 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1 0 1 1 0 0
 0 0 1 0 0 1 0 0 1 0 1 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0
 1 1 0 0 1 0 0 0 1 1 0 1 0 0 0 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0
 1 0 1 0 0 1 0 0 1 1 0 0 1 0 1 0 1 1 0 0 0 0 0 1 1 0 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0
 1 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 1 0 1 0 0 0 1 1 0
 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 0 1 1 1 1 0 0 0 0 0
 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 0 1 1 1 0 1 0 0
 0 1 1 1 1 0 0 1 0 0 0]


In [29]:
df_test['Survived'] = pd.Series(pred, index=df_test.index)

In [30]:
df_test

,PassengerId,Pclass,Sex,Age,Embarked,Family,Survived
0,892,3,1,34.5,3,1,0
1,893,3,0,47.0,1,2,0
2,894,2,1,62.0,3,1,1
3,895,3,1,27.0,1,1,1
4,896,3,0,22.0,1,3,1
...,...,...,...,...,...,...,...
413,1305,3,1,28.0,1,1,0
414,1306,1,0,39.0,2,1,1
415,1307,3,1,38.5,1,1,0
416,1308,3,1,28.0,1,1,0


In [31]:
final_df = df_test.filter(['PassengerId', 'Survived'], axis=1)

In [32]:
final_df.shape

(418, 2)

In [33]:
final_df.to_csv("pred_survival.csv", encoding='utf-8')

In [34]:
final_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
